In [1]:
import pandas as pd
import numpy as np
import requests
import os 
from dotenv import load_dotenv
import requests
from tqdm import tqdm
import pickle
import re
pd.options.display.max_columns = None

# Importing pickle file

In [2]:
with open('../data/pickle/dataset_viral150_unique.pickle', 'rb') as tracks:
    tracks = pickle.load(tracks)

In [3]:
tracks.head()

,url,artist_clean,track_clean
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert)
8247,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love
8248,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy
8249,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer
8250,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles


In [4]:
tracks.rename(columns={'track_clean':'track','artist_clean':'artist'}, inplace=True)
tracks.head()

,url,artist,track
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert)
8247,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love
8248,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy
8249,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer
8250,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles


# lastfm
In this part of the process I will extract form the LastFM API all the data related to the tracks and artist genres.

First, I need to configure my credentials.

In [5]:
load_dotenv()

True

In [6]:
api = os.getenv("lastfm-id")
username = os.getenv("lastfm-user")

Testing to gather the data from the api with a function:

In [7]:
## me voy a hacer una función con los datos de la petición
def getLastFMData(params):
    headers = {'user-agent': username} ## defino mi usuario
    url = 'https://ws.audioscrobbler.com/2.0/' ##url a la que voy a contactar

    ## configuro los params para hacer la petición
    params['api_key'] = api
    params['format'] = 'json'

    res = requests.get(url, headers=headers, params=params)
    return res

In [8]:
res = getLastFMData({
    'method': 'artist.getInfo',
    'artist':'Harry Styles',
})
res.json()['artist']['bio']

{'links': {'link': {'#text': '',
   'rel': 'original',
   'href': 'https://last.fm/music/Harry+Styles/+wiki'}},
 'published': '24 Aug 2011, 17:11',
 'summary': 'Harry Edward Styles (born 1 February 1994 in Redditch, Worcestershire, England) is a Grammy-winning, British singer, songwriter, and actor. As a member of the British (&Irish) boy band One Direction, singer Harry Styles topped the charts, toured the world, and sold millions of albums before going solo in 2016. Styles was raised in Holmes Chapel, Cheshire, and he made his first foray into music with a high-school band named White Eskimo. In 2010, he left home and joined the seventh season of The X Factor <a href="https://www.last.fm/music/Harry+Styles">Read more on Last.fm</a>',
 'content': 'Harry Edward Styles (born 1 February 1994 in Redditch, Worcestershire, England) is a Grammy-winning, British singer, songwriter, and actor. As a member of the British (&Irish) boy band One Direction, singer Harry Styles topped the charts, to

Function for gathering the artist info:

In [9]:
def getArtistInfo(artist):
    ## hago la colsulta para obtener los datos de tags por artista
    res = getLastFMData({
        'method': 'artist.getInfo',
        'artist':  artist,
    })

    # por si me da un timeout
    if res.status_code != 200:
        return None

    # extraigo los top 5 tags de cada artistas
    try:
        #tags = [tag['name'] for tag in res.json()['toptags']['tag'][:3]]
        return res.json()['artist']['bio'], res.json()['artist']['tags']['tag']
    except:
        return np.nan

First I try on a slice of my dataframe

In [10]:
tqdm.pandas()

test = tracks.sample(100)
test[['bio', 'artist_tag']] = test.progress_apply(lambda x: getArtistInfo(x.artist), axis=1, result_type='expand')

100%|██████████| 100/100 [00:21<00:00,  4.67it/s]


In [11]:
test.head()

,url,artist,track,bio,artist_tag
24676,https://open.spotify.com/track/4cxMGhkinTocPSV...,russ,losin control,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'Hip-Hop', 'url': 'https://www.last...."
318476,https://open.spotify.com/track/41A89rj3GoMG6kt...,lady gaga,alice,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'pop', 'url': 'https://www.last.fm/t..."
132642,https://open.spotify.com/track/5f5KGCBUtnR3dTS...,"jeremih, chris brown, big sean",i think of you,"{'links': {'link': {'#text': '', 'rel': 'origi...",[]
303376,https://open.spotify.com/track/6gxKUmycQX7uyMw...,lil baby,we paid (feat. 42 dugg),"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'Hip-Hop', 'url': 'https://www.last...."
7709923,https://open.spotify.com/track/6Rz7TXNHJpsyUs0...,carnage,"wait for me (feat. g-eazy, wiz khalifa & prinz...","{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'death metal', 'url': 'https://www.l..."


This is for the whole dataframe:

In [12]:
tqdm.pandas()

#test = tracks.sample(100)
tracks[['bio', 'artist_tag']] = tracks.progress_apply(lambda x: getArtistInfo(x.artist), axis=1, result_type='expand')

100%|██████████| 14659/14659 [41:20<00:00,  5.91it/s] 


In [13]:
tracks.head()

,url,artist,track,bio,artist_tag
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'Hip-Hop', 'url': 'https://www.last...."
8247,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'Hip-Hop', 'url': 'https://www.last...."
8248,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'mistagged', 'url': 'https://www.las..."
8249,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer,"{'links': {'link': {'#text': '', 'rel': 'origi...",[]
8250,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles,"{'links': {'link': {'#text': '', 'rel': 'origi...",[]


Now I get the tags for each individual track:

In [14]:
res = getLastFMData({
    'method': 'track.getInfo',
    'artist':'Harry Styles',
    'track':'As It Was'
})
res.json()['track']['toptags']['tag']#['published']#[0]['name']

[{'name': 'pop', 'url': 'https://www.last.fm/tag/pop'},
 {'name': 'rock', 'url': 'https://www.last.fm/tag/rock'},
 {'name': 'indie pop', 'url': 'https://www.last.fm/tag/indie+pop'},
 {'name': 'synthpop', 'url': 'https://www.last.fm/tag/synthpop'},
 {'name': 'british', 'url': 'https://www.last.fm/tag/british'}]

I create a function for gathering the track tags:

In [15]:
def getTrackTags(artist, track):
    ## hago la colsulta para obtener los datos de tags por artista
    res = getLastFMData({
        'method': 'track.getInfo',
        'artist':  artist,
        'track': track,
    })

    # por si me da un timeout
    if res.status_code != 200:
        return None

    # extraigo los top 5 tags de cada artistas
    try:
        #tags = [tag['name'] for tag in res.json()['toptags']['tag'][:3]]
        return res.json()['track']['toptags']['tag'], res.json()['track']['wiki']['published'] ## no va muy bien esta parte y solo me extrae tres géneros... 
    except:
        return np.nan, np.nan

This is how I will get info for a slice of my dataframe:

In [16]:
tqdm.pandas()

test = tracks.sample(100)
test[['track_tag', 'published_track']] = test.progress_apply(lambda x: getTrackTags(x.artist, x.track), axis=1, result_type='expand')

100%|██████████| 100/100 [00:18<00:00,  5.51it/s]


In [17]:
print(test['track_tag'].isnull().sum())
test.head()

62


,url,artist,track,bio,artist_tag,track_tag,published_track
2802690,https://open.spotify.com/track/3ig3X70suAwUn4o...,nicki minaj,sir (feat. future),"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'Hip-Hop', 'url': 'https://www.last....","[{'name': 'future', 'url': 'https://www.last.f...","05 Sep 2021, 01:01"
16165095,https://open.spotify.com/track/3MmvQKMKdN3UhVS...,pop smoke,brush em (feat. rah swish),"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'rap', 'url': 'https://www.last.fm/t...",NaN,NaN
24949571,https://open.spotify.com/track/0IIZah55ZZY6VEA...,"cl4pers, kingmichealbeats",want me (offcial) - 2021 tiktok version,"{'links': {'link': {'#text': '', 'rel': 'origi...",[],NaN,NaN
18714605,https://open.spotify.com/track/2JLVZYEMmqPWbUo...,jazmine sullivan,pick up your feelings,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'rnb', 'url': 'https://www.last.fm/t...",NaN,NaN
284066,https://open.spotify.com/track/2naJy8SKJ27gj6Z...,"rvssian, bad bunny, farruko, nicky jam, kosa",si tu lo dejas,"{'links': {'link': {'#text': '', 'rel': 'origi...",[],NaN,NaN


In [18]:
test.tail()

,url,artist,track,bio,artist_tag,track_tag,published_track
12035173,https://open.spotify.com/track/6dSKDp21S6I5UOh...,imagine dragons,mouth of the river,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'indie', 'url': 'https://www.last.fm...","[{'name': 'chill', 'url': 'https://www.last.fm...","28 Jan 2020, 18:20"
21309773,https://open.spotify.com/track/2158CKVDPFy5h5O...,run the jewels,stay gold,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'Hip-Hop', 'url': 'https://www.last....",NaN,NaN
23384226,https://open.spotify.com/track/0iI87VdRHdzSzX2...,conan gray,idle town,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'indie', 'url': 'https://www.last.fm...","[{'name': 'indie pop', 'url': 'https://www.las...","08 Apr 2022, 16:18"
23737723,https://open.spotify.com/track/02q0ZnV2L4XByzE...,bts,spring day,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'k-pop', 'url': 'https://www.last.fm...","[{'name': 'bts', 'url': 'https://www.last.fm/t...","19 Nov 2022, 19:36"
12491355,https://open.spotify.com/track/6Ww0J5EFjiPxkgZ...,eminem,nowhere fast (feat. kehlani),"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'rap', 'url': 'https://www.last.fm/t...",NaN,NaN


This is for the whole dataset:

In [19]:
tqdm.pandas()

tracks[['track_tag', 'published_track']] = tracks.progress_apply(lambda x: getTrackTags(x.artist, x.track), axis=1, result_type='expand')

100%|██████████| 14659/14659 [44:30<00:00,  5.49it/s] 


In [20]:
print(tracks['track_tag'].isnull().sum(), tracks['artist_tag'].isnull().sum())
tracks.head()

9858 82


,url,artist,track,bio,artist_tag,track_tag,published_track
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'Hip-Hop', 'url': 'https://www.last....","[{'name': 'trap', 'url': 'https://www.last.fm/...","30 Jan 2018, 04:41"
8247,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'Hip-Hop', 'url': 'https://www.last....","[{'name': 'trap', 'url': 'https://www.last.fm/...","27 Oct 2016, 02:34"
8248,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'mistagged', 'url': 'https://www.las...",NaN,NaN
8249,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer,"{'links': {'link': {'#text': '', 'rel': 'origi...",[],NaN,NaN
8250,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles,"{'links': {'link': {'#text': '', 'rel': 'origi...",[],NaN,NaN


# Data Cleaning

Cleaning the new `bio` data

In [21]:
bio = tracks['bio'].apply(pd.Series)
bio.head()
#artist_tag['tag'] = artist_tag['tag'].apply(pd.Series)

,links,published,summary,content,0
8246,"{'link': {'#text': '', 'rel': 'original', 'hre...","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",NaN
8247,"{'link': {'#text': '', 'rel': 'original', 'hre...","25 Mar 2006, 06:09","Aubrey Drake Graham (born October 24, 1986) is...","Aubrey Drake Graham (born October 24, 1986) is...",NaN
8248,"{'link': {'#text': '', 'rel': 'original', 'hre...","01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/The+Weeknd...",,NaN
8249,"{'link': {'#text': '', 'rel': 'original', 'hre...","01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/The+Chains...",,NaN
8250,"{'link': {'#text': '', 'rel': 'original', 'hre...","01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/Rae+Sremmu...",,NaN


Cleaning the `artist_tag` column

In [22]:
artist_tag = tracks['artist_tag'].apply(pd.Series)
new_columns = []
for item in artist_tag.columns.to_list():
    new_columns.append('aritist_genre_'+str(item))
artist_tag.columns = new_columns
#print(artist_tag.columns.to_list(), new_columns)
artist_tag.head()

C:\Users\annas\AppData\Local\Temp\ipykernel_23572\2079752897.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  artist_tag = tracks['artist_tag'].apply(pd.Series)


,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4
8246,"{'name': 'Hip-Hop', 'url': 'https://www.last.f...","{'name': 'trap', 'url': 'https://www.last.fm/t...","{'name': 'rap', 'url': 'https://www.last.fm/ta...","{'name': 'hip hop', 'url': 'https://www.last.f...","{'name': 'atlanta', 'url': 'https://www.last.f..."
8247,"{'name': 'Hip-Hop', 'url': 'https://www.last.f...","{'name': 'rap', 'url': 'https://www.last.fm/ta...","{'name': 'rnb', 'url': 'https://www.last.fm/ta...","{'name': 'hip hop', 'url': 'https://www.last.f...","{'name': 'Canadian', 'url': 'https://www.last...."
8248,"{'name': 'mistagged', 'url': 'https://www.last...","{'name': 'Daft Punk', 'url': 'https://www.last...","{'name': 'mistagged artist', 'url': 'https://w...","{'name': 'The Weeknd', 'url': 'https://www.las...",NaN
8249,NaN,NaN,NaN,NaN,NaN
8250,NaN,NaN,NaN,NaN,NaN


In [23]:
# cleaning unnecesary dictionaries
for index, row in artist_tag.iterrows():
    for column in artist_tag.columns.tolist():
        try:
            row[column] = list(row[column].values())[0] 
            #print(row[column])
        except:
            continue
artist_tag.head()

,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4
8246,Hip-Hop,trap,rap,hip hop,atlanta
8247,Hip-Hop,rap,rnb,hip hop,Canadian
8248,mistagged,Daft Punk,mistagged artist,The Weeknd,NaN
8249,NaN,NaN,NaN,NaN,NaN
8250,NaN,NaN,NaN,NaN,NaN


Cleaning `track_tag` column:

In [24]:
track_tag = tracks['track_tag'].apply(pd.Series)
new_columns = []
for item in track_tag.columns.to_list():
    new_columns.append('track_genre_'+str(item))
track_tag.columns = new_columns
track_tag.head()

C:\Users\annas\AppData\Local\Temp\ipykernel_23572\2338087104.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  track_tag = tracks['track_tag'].apply(pd.Series)


,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4
8246,"{'name': 'trap', 'url': 'https://www.last.fm/t...","{'name': 'Hip-Hop', 'url': 'https://www.last.f...","{'name': 'rap', 'url': 'https://www.last.fm/ta...","{'name': '2010s', 'url': 'https://www.last.fm/...","{'name': 'meme rap', 'url': 'https://www.last...."
8247,"{'name': 'trap', 'url': 'https://www.last.fm/t...","{'name': 'Hip-Hop', 'url': 'https://www.last.f...","{'name': 'rap', 'url': 'https://www.last.fm/ta...","{'name': 'hiphop', 'url': 'https://www.last.fm...","{'name': 'rnb', 'url': 'https://www.last.fm/ta..."
8248,NaN,NaN,NaN,NaN,NaN
8249,NaN,NaN,NaN,NaN,NaN
8250,NaN,NaN,NaN,NaN,NaN


In [25]:
# cleaning unnecesary dictionaries
for index, row in track_tag.iterrows():
    for column in track_tag.columns.tolist():
        try:
            row[column] = list(row[column].values())[0] 
        except:
            continue
track_tag.head()

,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4
8246,trap,Hip-Hop,rap,2010s,meme rap
8247,trap,Hip-Hop,rap,hiphop,rnb
8248,NaN,NaN,NaN,NaN,NaN
8249,NaN,NaN,NaN,NaN,NaN
8250,NaN,NaN,NaN,NaN,NaN


Merging the clean columns to the original dataframe and removing unnecesary columns:

In [27]:
tracks_final = pd.concat([tracks, bio, artist_tag, track_tag], axis = 1)
tracks_final.head()

,url,artist,track,bio,artist_tag,track_tag,published_track,links,published,summary,content,0,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'Hip-Hop', 'url': 'https://www.last....","[{'name': 'trap', 'url': 'https://www.last.fm/...","30 Jan 2018, 04:41","{'link': {'#text': '', 'rel': 'original', 'hre...","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",NaN,Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap
8247,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'Hip-Hop', 'url': 'https://www.last....","[{'name': 'trap', 'url': 'https://www.last.fm/...","27 Oct 2016, 02:34","{'link': {'#text': '', 'rel': 'original', 'hre...","25 Mar 2006, 06:09","Aubrey Drake Graham (born October 24, 1986) is...","Aubrey Drake Graham (born October 24, 1986) is...",NaN,Hip-Hop,rap,rnb,hip hop,Canadian,trap,Hip-Hop,rap,hiphop,rnb
8248,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'mistagged', 'url': 'https://www.las...",NaN,NaN,"{'link': {'#text': '', 'rel': 'original', 'hre...","01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/The+Weeknd...",,NaN,mistagged,Daft Punk,mistagged artist,The Weeknd,NaN,NaN,NaN,NaN,NaN,NaN
8249,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer,"{'links': {'link': {'#text': '', 'rel': 'origi...",[],NaN,NaN,"{'link': {'#text': '', 'rel': 'original', 'hre...","01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/The+Chains...",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8250,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles,"{'links': {'link': {'#text': '', 'rel': 'origi...",[],NaN,NaN,"{'link': {'#text': '', 'rel': 'original', 'hre...","01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/Rae+Sremmu...",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
tracks_final.drop(['bio', 'artist_tag', 'track_tag', 'links', 0], axis=1, inplace=True)
tracks_final.head()

,url,artist,track,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap
8247,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love,"27 Oct 2016, 02:34","25 Mar 2006, 06:09","Aubrey Drake Graham (born October 24, 1986) is...","Aubrey Drake Graham (born October 24, 1986) is...",Hip-Hop,rap,rnb,hip hop,Canadian,trap,Hip-Hop,rap,hiphop,rnb
8248,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy,NaN,"01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/The+Weeknd...",,mistagged,Daft Punk,mistagged artist,The Weeknd,NaN,NaN,NaN,NaN,NaN,NaN
8249,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer,NaN,"01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/The+Chains...",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8250,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles,NaN,"01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/Rae+Sremmu...",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Export

In [ ]:
with open('../data/pickle/spotify_dataset_lastfm.pickle', 'wb') as data_lastfm:
    pickle.dump(tracks_final, data_lastfm)